In [10]:
import urllib2
from collections import Counter
response = urllib2.urlopen('http://erdani.com/tdpl/hamlet.txt')
text_to_encode = response.read()

In [11]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
        
    def children(self):
        c = []
        if self.left:
            c.append(self.left)
        if self.right:
            c.append(self.right)
        return c
    
    def __str__(self):
        return "{}({})".format(self.key, self.value)

In [12]:
def nodify(items):
    return [Node(k, v) for k, v in items]

def build_node(n1, n2):
    n = Node("{}_{}".format(n1.key, n2.key), n1.value + n2.value)
    n.left = n1
    n.right = n2
    return n

def sorted_insert(node, nodes):
    nodes.append(node)
    nodes = sorted(nodes, key=lambda n: n.value)
    return nodes
    
def print_nodes(nodes):
    for n in nodes:
        print n,
    print
        
def build_huffman_tree(counter):
    keys = counter.keys()
    items = counter.most_common()
    items.reverse()
    nodes = nodify(items)
    
    while len(nodes) != 1:
        #print_nodes(nodes)
        n1 = nodes.pop(0)
        n2 = nodes.pop(0)
        new_node = build_node(n1, n2)
        nodes = sorted_insert(new_node, nodes) 
    return nodes[0]

In [13]:
def encoding_for_key(key, root):
    if root.key == key:
        return ''
    
    left = root.left
    right = root.right
    if left:
        ret_val = encoding_for_key(key, left)
        if type(ret_val) == str:
            return '0' + ret_val

    if right:
        ret_val = encoding_for_key(key, right)
        if type(ret_val) == str:
            return '1' + ret_val
    return False

def build_table(keys, root):
    table = {}
    for k in keys:
        print("{}: {}".format(k, encoding_for_key(k, root)))
        table[k] = encoding_for_key(k, root)
    return table

In [16]:
def encode(text_to_encode):
    # Count the characters in the text
    chars = list(text_to_encode)
    counter = Counter(chars)
    
    # Build huffman tree from counts
    root = build_huffman_tree(counter)
    
    # Build table for each character in counter
    table = build_table(counter.keys(), root)
    encoding = ''.join([table[c] for c in chars])
    return encoding, table

encoding, table = encode(text_to_encode)
original_size = len(text_to_encode) * 8
encoded_size = len(encoding)
print "Original Data: {} bits".format(original_size)
print "Encoded Data: {} bits".format(encoded_size)
print "Achieved a {}% compression".format(int(float(encoded_size) / original_size * 100))


: 00010
!: 011001100
 : 10
": 0000000011001010
': 0001111
&: 0000000011001001
): 000110111110
(: 000110111111
-: 000110110
,: 011100
.: 011110
1: 00000000110011
0: 0000000011001000
4: 00000000110010111
6: 00000000110010110
;: 011111101
:: 000000001101
?: 110100110
A: 01100100
C: 0110010100
B: 000110100
E: 000000000
D: 0001101011
G: 000000101
F: 0111111000
I: 01111111
H: 11010010
K: 0110010111
J: 0000000011000
M: 000000110
L: 000000100
O: 011001101
N: 0110010110
Q: 11010011101
P: 1101001111
S: 000000111
R: 0001101110
U: 000000001111
T: 01100111
W: 00000001
V: 000000001110
Y: 0001101010
[: 0000000010
]: 11010011100
a: 0010
c: 001110
b: 0111110
e: 1100
d: 00110
g: 000001
f: 001111
i: 11100
h: 11101
k: 0001100
j: 01111110010
m: 110101
l: 01101
o: 0100
n: 11110
q: 01111110011
p: 1101000
s: 11111
r: 11011
u: 00001
t: 0101
w: 011000
v: 0001110
y: 011101
x: 0110010101
z: 00011011110
Original Data: 1533872 bits
Encoded Data: 843328 bits
Achieved a 54% compression


In [17]:
def decode(encoding, table):
    reverse_table = { v:k for k, v in table.items() }
    str_so_far = ''
    decoding = ''
    for b in encoding: 
        str_so_far += b
        if str_so_far in reverse_table.keys():
            decoding += reverse_table[str_so_far]
            str_so_far = ''
    return decoding 
decoding = decode(encoding, table)

In [18]:
assert decoding == text_to_encode
print("Decoded input is correct!")

Decoded input is correct!
